# Qubit Dynamicas

In [1]:
import numpy as np
import qutip as qt
import qctrlvisualizer as qv


def simulate_one_qubit(hamiltonian, times, initial_state, args={}):
    initial_state = qt.Qobj(initial_state)
    propagators = [qt.propagator(hamiltonian, t, args=args) for t in times]
    states = [U * initial_state for U in propagators]
    states = np.array(states).squeeze()
    qv.display_bloch_sphere(states)


def simulate_two_qubits(hamiltonian, times, initial_state_0, initial_state_1, args={}):
    initial_state = qt.tensor(initial_state_0, initial_state_1)
    propagators = [qt.propagator(hamiltonian, t, args=args) for t in times]
    states = [U * initial_state for U in propagators]
    density_matrices = [qt.ket2dm(state) for state in states]
    density_matrices_0 = np.array([dm.ptrace(0) for dm in density_matrices])
    density_matrices_1 = np.array([dm.ptrace(1) for dm in density_matrices])
    qv.display_bloch_sphere_from_density_matrices(density_matrices_0)
    qv.display_bloch_sphere_from_density_matrices(density_matrices_1)

## 1-qubit

$$
H = \frac{\omega}{2} \sigma_z
$$

In [2]:
omega = 2 * np.pi
H = 0.5 * omega * qt.sigmaz()
times = np.linspace(0, 1, 20)

initial_state = 1 / np.sqrt(2) * (qt.basis(2, 0) + qt.basis(2, 1))  # |+>
# initial_state = np.sqrt(3) / 2 * qt.basis(2, 0) + 1 / 2 * qt.basis(2, 1)
# initial_state = qt.rand_ket_haar(2)

simulate_one_qubit(H, times, initial_state)

<IPython.core.display.Javascript object>

$$
\Omega = |\Omega| e^{i\phi}
$$

$$
\begin{aligned}
\sigma_+ &= \frac{\sigma_x - i\sigma_y}{2}
\\
\sigma_- &= \frac{\sigma_x + i\sigma_y}{2}
\end{aligned}
$$

$$
\begin{aligned}
H &= \frac{1}{2}\left( \Omega \sigma_+ + \Omega^* \sigma_- \right)
\\
&= \frac{1}{2}\left( |\Omega| e^{i\phi} \frac{\sigma_x - i\sigma_y}{2} + |\Omega| e^{-i\phi} \frac{\sigma_x + i\sigma_y}{2} \right)
\\
&= \frac{1}{2} |\Omega| \left( e^{i\phi} \frac{\sigma_x - i\sigma_y}{2} + e^{-i\phi} \frac{\sigma_x + i\sigma_y}{2} \right)
\\
&= \frac{1}{2} |\Omega| \left( \frac{e^{i\phi} + e^{-i\phi}}{2} \sigma_x + \frac{e^{i\phi} - e^{-i\phi}}{2i} \sigma_y \right)
\\
&= \frac{1}{2} |\Omega| \left( \cos\phi \sigma_x + \sin\phi \sigma_y \right)
\\
&= \frac{1}{2} \left( |\Omega| \cos\phi \sigma_x + |\Omega| \sin\phi \sigma_y \right)
\\
&= \frac{1}{2} \left( I \sigma_x + Q \sigma_y \right)
\end{aligned}
$$


In [3]:
I = np.pi
H = 0.5 * I * qt.sigmax()
times = np.linspace(0, 1, 20)

initial_state = qt.basis(2, 0)  # |0>

simulate_one_qubit(H, times, initial_state)

<IPython.core.display.Javascript object>

### Time-dependent Hamiltonian

$$
\begin{aligned}
H &= \frac{I(t)}{2} \sigma_x
\\
I(t) &= \frac{\omega}{\sqrt{2\pi}\sigma}  e^{-\frac{(t - \mu)^2}{\sigma^2}}
\end{aligned}
$$

#### String-based

In [4]:
X = qt.sigmax()
T = 1
mu = 0.5 * T
sigma = T / 6

hamiltonian = [
    [
        X,
        "0.5 * omega / (sqrt(2 * pi) * sigma) * exp(-((t - mu) ** 2) / (2 * sigma ** 2))",
    ],
]
args = {
    "mu": mu,
    "sigma": sigma,
    "omega": np.pi,
}
times = np.linspace(0, T, 20)
initial_state = qt.basis(2, 0)
simulate_one_qubit(hamiltonian, times, initial_state, args)

<IPython.core.display.Javascript object>

#### Function-based

In [5]:
X = qt.sigmax()
T = 1.0

args = {
    "mu": 0.5 * T,
    "sigma": T / 6,
    "omega": np.pi,
}


def X_coeff(t, args):
    return (
        0.5
        * args["omega"]
        / (np.sqrt(2 * np.pi) * args["sigma"])
        * np.exp(-((t - args["mu"]) ** 2) / (2 * args["sigma"] ** 2))
    )


hamiltonian = [
    [X, X_coeff],
]
times = np.linspace(0, T, 20)
initial_state = qt.basis(2, 0)
simulate_one_qubit(hamiltonian, times, initial_state, args)

<IPython.core.display.Javascript object>

## 2-qubits

### ZX Hamiltonian

$$
H = \frac{1}{2} \sigma_z \otimes \sigma_x
$$

In [6]:
H = 0.5 * qt.tensor(qt.sigmaz(), qt.sigmax())
times = np.linspace(0, np.pi, 20)

$$\ket{0}\otimes\ket{0}$$

In [7]:
initial_state_0 = qt.basis(2, 0)
initial_state_1 = qt.basis(2, 0)

simulate_two_qubits(H, times, initial_state_0, initial_state_1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

$$\ket{1}\otimes\ket{0}$$

In [8]:
initial_state_0 = qt.basis(2, 1)
initial_state_1 = qt.basis(2, 0)

simulate_two_qubits(H, times, initial_state_0, initial_state_1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### ZZ Hamiltonian

$$
H = \frac{1}{2} \sigma_z \otimes \sigma_z
$$

In [9]:
H = 0.5 * qt.tensor(qt.sigmaz(), qt.sigmaz())
times = np.linspace(0, np.pi, 20)

$$\ket{0}\otimes\ket{+}$$

In [10]:
initial_state_0 = qt.basis(2, 0)
initial_state_1 = 1 / np.sqrt(2) * (qt.basis(2, 0) + qt.basis(2, 1))

simulate_two_qubits(H, times, initial_state_0, initial_state_1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

$$\ket{+}\otimes\ket{+}$$

In [11]:
H = 0.5 * qt.tensor(qt.sigmaz(), qt.sigmaz())
times = np.linspace(0, np.pi, 20)

# initial_state_0 = qt.rand_ket_haar(2)
# initial_state_0 = qt.basis(2, 0)
# initial_state_0 = qt.basis(2, 1)
initial_state_0 = 1 / np.sqrt(2) * (qt.basis(2, 0) + qt.basis(2, 1))
# initial_state_0 = np.sqrt(3) / 2 * qt.basis(2, 0) + 1 / 2 * qt.basis(2, 1)

# initial_state_1 = qt.rand_ket_haar(2)
# initial_state_1 = qt.basis(2, 0)
# initial_state_1 = qt.basis(2, 1)
initial_state_1 = 1 / np.sqrt(2) * (qt.basis(2, 0) + qt.basis(2, 1))
# initial_state_1 = np.sqrt(3) / 2 * qt.basis(2, 0) + 1 / 2 * qt.basis(2, 1)

simulate_two_qubits(H, times, initial_state_0, initial_state_1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>